In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import pygraphviz
from pandas import read_csv
import seaborn as sns
import lightgbm as lgb

from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier

from lightgbm import LGBMClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
import os



In [2]:
from data.utils.HGTDB.ml_data_loader import ml_load_species

In [3]:
X_ecoli_A, y_ecoli_A = ml_load_species('ecoli', 'A')
X_ecoli_B, y_ecoli_B = ml_load_species('ecoli', 'B')
X_ecoli_C, y_ecoli_C = ml_load_species('ecoli', 'C')
X_ecoli_D, y_ecoli_D = ml_load_species('ecoli', 'D')
X_ecoli_E, y_ecoli_E = ml_load_species('ecoli', 'E')
print(X_ecoli_A.shape)
print(X_ecoli_B.shape)
print(X_ecoli_C.shape)
print(X_ecoli_D.shape)
print(X_ecoli_E.shape)

(4276, 13)
(4276, 12)
(4276, 10)
(4276, 4)
(4276, 8)


In [4]:
from trainer.ml_trainer import ml_trainer

In [5]:
ecoli_B_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_B_lgbm,'ecoli','B',cv,'average_precision',False)

**** Training on ecoli with type B ****

average_precision: 0.611 (0.065)
[[776  29]
 [ 23  28]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.49      0.55      0.52        51

    accuracy                           0.94       856
   macro avg       0.73      0.76      0.74       856
weighted avg       0.94      0.94      0.94       856



In [6]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [7]:
from data.utils.HGTDB.dl_data_loader import SingleSpecies

In [8]:
test_single = SingleSpecies(species="bsub",data_type="B")

test_single[500]

{'gc_signature': array([ 0.000e+00,  1.173e+03,  3.980e+01, -2.200e+00,  2.450e+01,
        -2.200e+00,  2.810e+01, -2.100e+00,  3.080e+01, -3.000e+00,
         1.490e+01,  1.000e+00]),
 'hgt': 1.0}

In [9]:
from data.utils.HGTDB.dl_data_loader import AllSpecies

In [10]:
test_all = AllSpecies(data_type="B")

test_all[500]

{'gc_signature': tensor([  0.0000, 387.0000,  56.2000,   0.8000,  38.5000,   0.9000,  26.2000,
          -1.2000,  40.3000,   0.0000,  70.8000,   1.0000], dtype=torch.float64),
 'hgt': tensor(0., dtype=torch.float64)}

In [11]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 100).double()
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(100, 1).double()
        self.sigmoid1 = torch.nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.sigmoid1(x)
        return x

In [14]:
from trainer.dl_trainer import DLClassifier

In [15]:
b_holdout_train_set = AllSpecies('B',normalize=False, partition_type='holdout', partition='train')
b_holdout_test_set = AllSpecies('B',normalize=False, partition_type='holdout', partition='test')
bacteria_classifier_B = DLClassifier(Feedforward(12), torch.nn.BCELoss(), torch.optim.Adam, 0.001)

bacteria_classifier_B.train(b_holdout_train_set, 16, 10)

EPOCH 1:
LOSS train 0.39401663493158723
EPOCH 2:
LOSS train 0.3002877629044
EPOCH 3:
LOSS train 0.2577981494156747
EPOCH 4:
LOSS train 0.2271811320152321
EPOCH 5:
LOSS train 0.2179799544590502
EPOCH 6:
LOSS train 0.2117221989823492
EPOCH 7:
LOSS train 0.20662721559992359
EPOCH 8:
LOSS train 0.19963526992741173
EPOCH 9:
LOSS train 0.19359634094590297
EPOCH 10:
LOSS train 0.1906631274738119


In [1]:
from data.utils.NCBI.data_loader import NewGenes

In [3]:
yakult = NewGenes('ASM82905v1')

found 1 ids
ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/829/055/GCF_000829055.1_ASM82905v1/GCF_000829055.1_ASM82905v1_cds_from_genomic.fna.gz


In [4]:
print(yakult.complete_sequence)

...


In [5]:
print(yakult.mean_GCT)
print(yakult.mean_GC1)
print(yakult.mean_GC2)
print(yakult.mean_GC3)
print(yakult.std_GCT)
print(yakult.std_GC1)
print(yakult.std_GC2)
print(yakult.std_GC3)

47.99780371380697
54.43022578518216
38.120079456762745
51.4432288224944
3.9333303002550197
5.59620581567793
5.1420556965271995
7.1901061819491865
